In [1]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
labels = ["drowsiness" , "undrowsiness"]
model = tf.keras.models.load_model('best_so_far.h5')
print(model.input.shape)

(None, 150, 150, 3)


In [5]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)      

In [4]:
def get_label_index(pre):
    if pre[0][0] > 0.5:
        return 1
    else:
        return 0

In [16]:
import numpy as np
cap = cv2.VideoCapture("test/video/aslam_drowsiness.mp4")
label = "TESTTEST"
TRACKER = []
count = 0
size = 150
count = 0
labels = ["drowsiness" , "undrowsiness"]
face_cascade = cv2.CascadeClassifier("haarcascade/haarcascade_frontalface_alt.xml")
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
#     frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    width,height,layer = frame.shape
    
    # Our operations on the frame come here
#     frame = cv2.fastNlMeansDenoisingColored(frame,None,10,10,7,11)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,minSize = (30,30))
    for (x,y,w,h) in faces:
#         print(x,y,w,h)
        roi_image = frame[y:y+h,x:x+w]
        
        roi_image = cv2.resize(roi_image , (size , size))
        roi_image = np.array(roi_image,dtype="float64")
        roi_image = roi_image.reshape(-1,size,size,3)
#         roi_image = np.array(roi_image,dtype="float32")
        pre = model.predict([roi_image])
    
        color = (0,211,255)
        stroke = 2
        end_x = x+w
        end_y = y+h
        
        cv2.rectangle(frame, (x,y), (end_x,end_y), color, stroke)
        label_index = get_label_index(pre)
        
        TRACKER.append(label_index)
        
        if len(TRACKER)==24:
            total_sum = sum(TRACKER)
            avg = total_sum/24
            if avg > 0.5:
                label = "undrowsiness"
            else:
                label = "drowsiness"
                
        print(label_index,pre)

    cv2.putText(frame,label, (width-100,height-100), cv2.FONT_HERSHEY_SIMPLEX, 3, (50,200,255))
    frame = cv2.resize(frame,(800,600))
    cv2.imshow('frame',frame)
    count+=1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


1 [[0.88640547]]
1 [[0.9486356]]
1 [[0.97854054]]
